# Notebook 02 - TRANSFORMAÇÃO DE DADOS
# Pipeline de Big Data - Análise de Churn Telecom
# Equipe: [NOMES DOS MEMBROS]

## 🎯 Objetivo
Este notebook implementa a etapa de **Transformação** do pipeline de Big Data, convertendo dados brutos (Bronze) em dados limpos e estruturados (Silver).

## 📋 Etapas:
1. Carregamento dos dados brutos
2. Limpeza e validação
3. Engenharia de features
4. Normalização e padronização
5. Salvamento na camada Silver


In [ ]:
# ============================================================
# ETAPA 1: CONFIGURAÇÃO DO AMBIENTE
# ============================================================

import pandas as pd
import numpy as np
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

# Bibliotecas para análise
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar visualizações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Ambiente configurado com sucesso!")
print(f"📅 Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
# ============================================================
# ETAPA 2: CARREGAMENTO DOS DADOS BRUTOS (BRONZE)
# ============================================================

print("\n" + "="*60)
print("📥 CARREGANDO DADOS DA CAMADA BRONZE")
print("="*60)

# Carregar dados brutos
caminho_bronze = 'dados/bronze/telco_churn_raw.csv'

if os.path.exists(caminho_bronze):
    df_bronze = pd.read_csv(caminho_bronze)
    print(f"✅ Dados carregados com sucesso!")
    print(f"   📊 Registros: {len(df_bronze):,}")
    print(f"   📋 Colunas: {df_bronze.shape[1]}")
else:
    raise FileNotFoundError(f"❌ Arquivo não encontrado: {caminho_bronze}")

# Criar cópia para transformação
df_silver = df_bronze.copy()
print(f"\n🔄 Cópia criada para transformação")
print(f"   📊 Registros na cópia: {len(df_silver):,}")


In [ ]:
# ============================================================
# ETAPA 3: LIMPEZA E VALIDAÇÃO DOS DADOS
# ============================================================

print("\n" + "="*60)
print("🧹 LIMPEZA E VALIDAÇÃO DOS DADOS")
print("="*60)

# 1. Verificar e tratar valores ausentes
print("\n1️⃣ Tratamento de valores ausentes:")
missing_before = df_silver.isnull().sum().sum()
print(f"   Valores ausentes antes: {missing_before}")

# Verificar TotalCharges (pode ter valores vazios)
if 'TotalCharges' in df_silver.columns:
    # Converter TotalCharges para numérico, tratando espaços vazios
    df_silver['TotalCharges'] = pd.to_numeric(df_silver['TotalCharges'], errors='coerce')
    
    # Preencher valores ausentes com 0 (clientes novos)
    df_silver['TotalCharges'].fillna(0, inplace=True)
    print(f"   ✅ TotalCharges convertido e valores ausentes tratados")

missing_after = df_silver.isnull().sum().sum()
print(f"   Valores ausentes depois: {missing_after}")

# 2. Verificar duplicatas
print("\n2️⃣ Verificação de duplicatas:")
duplicatas = df_silver.duplicated(subset='customerID').sum()
print(f"   Duplicatas encontradas: {duplicatas}")
if duplicatas > 0:
    df_silver = df_silver.drop_duplicates(subset='customerID')
    print(f"   ✅ Duplicatas removidas")

print(f"\n✅ Limpeza concluída!")
print(f"   📊 Registros finais: {len(df_silver):,}")


In [ ]:
# ============================================================
# ETAPA 4: ENGENHARIA DE FEATURES
# ============================================================

print("\n" + "="*60)
print("⚙️ ENGENHARIA DE FEATURES")
print("="*60)

# 1. Codificação da variável alvo
print("\n1️⃣ Codificação da variável alvo:")
df_silver['Churn_numeric'] = (df_silver['Churn'] == 'Yes').astype(int)
print(f"   ✅ Churn convertido para numérico (0/1)")
print(f"   📊 Distribuição: {df_silver['Churn_numeric'].value_counts().to_dict()}")

# 2. Criação de features categóricas
print("\n2️⃣ Criação de features categóricas:")

# Codificar variáveis categóricas binárias
binary_columns = ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
for col in binary_columns:
    if col in df_silver.columns:
        df_silver[f'{col}_encoded'] = (df_silver[col] == 'Yes').astype(int)
        print(f"   ✅ {col} codificado")

# Codificar InternetService
if 'InternetService' in df_silver.columns:
    internet_mapping = {'No': 0, 'DSL': 1, 'Fiber optic': 2}
    df_silver['InternetService_encoded'] = df_silver['InternetService'].map(internet_mapping)
    print(f"   ✅ InternetService codificado")

# Codificar Contract
if 'Contract' in df_silver.columns:
    contract_mapping = {'Month-to-month': 0, 'One year': 1, 'Two year': 2}
    df_silver['Contract_encoded'] = df_silver['Contract'].map(contract_mapping)
    print(f"   ✅ Contract codificado")

# 3. Criação de features numéricas derivadas
print("\n3️⃣ Criação de features numéricas:")

# Feature: Total de serviços contratados
service_columns = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
                  'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

df_silver['TotalServices'] = 0
for col in service_columns:
    if col in df_silver.columns:
        df_silver['TotalServices'] += (df_silver[col] == 'Yes').astype(int)

print(f"   ✅ TotalServices criado")
print(f"   📊 Range: {df_silver['TotalServices'].min()} a {df_silver['TotalServices'].max()} serviços")

# Feature: Valor médio por mês de permanência
df_silver['AvgMonthlyCharges'] = df_silver['TotalCharges'] / (df_silver['tenure'] + 1)
print(f"   ✅ AvgMonthlyCharges criado")

# Feature: Cliente novo (tenure <= 1)
df_silver['IsNewCustomer'] = (df_silver['tenure'] <= 1).astype(int)
print(f"   ✅ IsNewCustomer criado")

# Feature: Cliente de longo prazo (tenure >= 36)
df_silver['IsLongTermCustomer'] = (df_silver['tenure'] >= 36).astype(int)
print(f"   ✅ IsLongTermCustomer criado")

print(f"\n✅ Engenharia de features concluída!")
print(f"   📊 Total de colunas: {df_silver.shape[1]}")


In [ ]:
# ============================================================
# ETAPA 5: SALVAMENTO NA CAMADA SILVER
# ============================================================

print("\n" + "="*60)
print("💾 SALVAMENTO NA CAMADA SILVER")
print("="*60)

# Adicionar metadados de transformação
df_silver['data_transformacao'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_silver['versao_transformacao'] = '1.0'

# Salvar em CSV
caminho_silver_csv = 'dados/silver/telco_churn_transformed.csv'
df_silver.to_csv(caminho_silver_csv, index=False)
print(f"✅ Dados salvos em CSV: {caminho_silver_csv}")

# Salvar em Parquet (mais eficiente)
try:
    import pyarrow as pa
    import pyarrow.parquet as pq
    
    caminho_silver_parquet = 'dados/silver/telco_churn_transformed.parquet'
    df_silver.to_parquet(caminho_silver_parquet, index=False)
    print(f"✅ Dados salvos em Parquet: {caminho_silver_parquet}")
    
    # Comparar tamanhos
    size_csv = os.path.getsize(caminho_silver_csv) / 1024**2
    size_parquet = os.path.getsize(caminho_silver_parquet) / 1024**2
    compression_ratio = (1 - size_parquet/size_csv) * 100
    
    print(f"\n📊 Comparação de tamanhos:")
    print(f"   - CSV: {size_csv:.2f} MB")
    print(f"   - Parquet: {size_parquet:.2f} MB")
    print(f"   - Compressão: {compression_ratio:.1f}%")
    
except ImportError:
    print("⚠️ PyArrow não disponível. Apenas CSV salvo.")

# Criar relatório de transformação
new_features = [col for col in df_silver.columns if col not in df_bronze.columns]
relatorio_transformacao = f"""
RELATÓRIO DE TRANSFORMAÇÃO DE DADOS
Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
==========================================
Fonte: dados/bronze/telco_churn_raw.csv
Registros processados: {len(df_silver):,}
Colunas finais: {df_silver.shape[1]}
Features criadas: {len(new_features)}
Valores ausentes: {df_silver.isnull().sum().sum()}
Taxa de Churn: {(df_silver['Churn_numeric']==1).mean()*100:.2f}%
==========================================
Features criadas:
{chr(10).join([f'- {feature}' for feature in new_features])}
==========================================
"""

with open('dados/silver/relatorio_transformacao.txt', 'w', encoding='utf-8') as f:
    f.write(relatorio_transformacao)

print(f"\n📄 Relatório de transformação salvo em: dados/silver/relatorio_transformacao.txt")

print(f"\n" + "="*60)
print("✅ TRANSFORMAÇÃO COMPLETA - Dados prontos para análise!")
print("="*60)

print(f"\n🎯 Próximos passos:")
print(f"   - Execute o notebook de análise (03_analise_visualizacao.ipynb)")
print(f"   - Os dados transformados estão na camada Silver!")
